In [69]:
# !pip3 install GoogleNews
# !pip3 install prettytable
from GoogleNews import GoogleNews
import pandas as pd
from datetime import datetime
from prettytable import PrettyTable
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


sender_email = "xinxianwang21@gmail.com"
receiver_email = "xinxianwang21@gmail.com"
password = "ktzw aouc zgev bjgp"
server_str = 'smtp.gmail.com'
port_num = 587


In [72]:
today = datetime.now()+ pd.Timedelta(days=1)
googlenews = GoogleNews()
googlenews.enableException(True)
googlenews = GoogleNews(lang='en', region='US')
yesterday = today - pd.Timedelta(days=2)
week_ago = today - pd.Timedelta(days=8)
today_dt = today.strftime('%m/%d/%Y')
yesterday_dt = yesterday.strftime('%m/%d/%Y')
week_ago_dt = week_ago.strftime('%m/%d/%Y')

googlenews_1d = GoogleNews(start=yesterday_dt, end=today_dt)
googlenews_1d.get_news('Federal Reserve')
result_1d=googlenews_1d.result(sort=True)

googlenews_7d = GoogleNews(start=week_ago_dt, end=yesterday_dt)
googlenews_7d.get_news('Federal Reserve')
result_7d=googlenews_7d.result(sort=True)

df_1d=pd.DataFrame(result_1d)
df_7d=pd.DataFrame(result_7d)
# df['media']

# pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_colwidth')
pd.set_option('display.max_row', None)
media_sources = ['Financial Times', 'Bloomberg', 'The Wall Street Journal', 'The New York Times']
pattern = '|'.join(media_sources)
df_1d = df_1d[df_1d['media'].str.contains(pattern, case=False)]
df_1d = df_1d.drop_duplicates(subset=['title'])
df_1d = df_1d[['title', 'date', 'media', 'link']]

In [73]:
df_7d = df_7d[df_7d['media'].str.contains(pattern, case=False)]
df_7d = df_7d.drop_duplicates(subset=['title'])
df_7d = df_7d[['title', 'date', 'media', 'link']]
df_7d

,title,date,media,link
0,"Nobel Laureate Romer on US Economy, Federal Re...",Yesterday,Bloomberg,news.google.com/articles/CBMiZmh0dHBzOi8vd3d3L...
2,Fed's Kashkari Says Rate Hike Not Entirely Rul...,2 days ago,Bloomberg,news.google.com/articles/CBMieWh0dHBzOi8vd3d3L...
9,Mester Says Fed Can Better Explain Officials’ ...,2 days ago,Bloomberg,news.google.com/articles/CBMiggFodHRwczovL3d3d...
31,Opinion | Is the Fed Waiting Too Long to Cut I...,2 days ago,The New York Times,news.google.com/articles/CBMiT2h0dHBzOi8vd3d3L...
43,"Treasury Buybacks Offer Support, Bond Traders ...",4 days ago,Bloomberg,news.google.com/articles/CBMifGh0dHBzOi8vd3d3L...
56,Goldman Axes Bet on July Fed Cut as Market See...,6 days ago,Bloomberg,news.google.com/articles/CBMieWh0dHBzOi8vd3d3L...
80,FirstFT: Fed officials were open to raising ra...,7 days ago,Financial Times,news.google.com/articles/CBMiP2h0dHBzOi8vd3d3L...
89,Fed's Bostic Says Policy Is Taking Longer to S...,7 days ago,Bloomberg,news.google.com/articles/CBMia2h0dHBzOi8vd3d3L...
99,What Trump 2.0 Could Mean for the Federal Reserve,7 days ago,The New York Times,news.google.com/articles/CBMiTmh0dHBzOi8vd3d3L...


In [74]:
def df_to_html_with_styles(df):
    if df.empty:
      return "<p>No news update</p>"
    else:
      df.style.set_properties(subset=['title'], **{'width': '3000px'})
      html = df.to_html(index=False, escape=False)
      
      html = html.replace('<th>', '<th style="text-align: left;">')
      html = html.replace('<td>', '<td align="left">')
      html = html.replace('<td style="width: 300px;">', '<td style="width: 300px;"><b>').replace('</td>', '</b></td>', 1)
      return html

html = f"""\
<html>
  <head></head>
  <body>
    <h2>Past 24 Hours:</h2>
    {df_to_html_with_styles(df_1d)}
    <br>
    <h2>Older (7d):</h2>
    {df_to_html_with_styles(df_7d)}
  </body>
</html>
"""

msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = "Fed News Alert"

part1 = MIMEText(html, 'html')
msg.attach(part1)

# Send email
try:
    server = smtplib.SMTP(server_str, port_num)
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()
    print("Email sent successfully")
except Exception as e:
    print(f"Failed to send email: {e}")


Email sent successfully
